# Geneva house renting consultancy.
Capstone Project, Applied Data Science Capstone by IBM/Coursera, Simone Lisi

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusions](#conclusions)




## 1. Introduction. <a name="introduction"></a>

In this project, we will investigate the trends in the housing rental market of the city of 
Geneva, Switzerland. Our customers will be newcomers looking for advice on renting an apartment. 

Switzerland is among the richest countries in the world. According to a 2018 study by the Swiss financial institution UBS <a href="https://www.ubs.com/microsites/prices-earnings/en/" target="_top">on the most expensive cities in the world</a>, Geneva and Zurich, both in Switzerland, ranked as the top two. The high cost of life in these cities is normally compensated by the high average salaries paid by employers. However, optimizing expenses still plays a key role in people’s well-beings because rental costs remains high in Geneva. 

In fact, rent is typically the single largest cost for most people. That is why optimizing one’s monthly rent is crucial to improving budget management. Optimizing in this context means answering 2 key questions:

* Which are the features that influence the price of a house the most?
* Are there external factors determining the price difference between properties with nominally identical features?

Certainly, there are obvious factors that influence the market value of a residential property directly, e.g. the surface area, or number of rooms. The influence of these kind of features on the rent will be carried out quantitatively. In some cases, two seemingly identical properties could have sensibly different prices. There could be several reasons for this, namely the more expensive property could be surrounded by better services. Or it could simply be in a more prestigious neighborhood. Other reasons could be that the less expensive property has been out of the market for an extended period of the time and avoided the price rise that lead to its current expected price. 

Answering these questions will help our customers better understand what they are really paying for. Whether it is extra space or better accessibility to certain services, these informations will enable customers to make more educated choices and find the apartments that suits them best. 


## 2. Data. <a name="data"></a>

In this section we will see the fundamental steps of the data collection. 
To carry out our study, the data have been scraped from two main sources:
* A popular Swiss rental listing website <a href="https://www.immoscout24.ch/en" target="_top">immoscout24.ch.</a>
* <a href="https://foursquare.com" target="_top">The foursquare database.</a>





### Scraping immoscout24.ch.

A large part of the housing market in Geneva can be found online through rental listing websites, which offers arguably the best visibility on the market. The first part of our project revolved around collecting all the currently available leases (29/06/2020) on the <a href="https://www.immoscout24.ch/en" target="_top">immoscout24.ch</a> website.
The source code for this analysis can be found in the 'ScrapingImmoscout24ch_FA.ipynb' notebook. 
A comprehensive set of features can be obtained for each residential property, e.g. rental price, number of rooms, surface area and location. 

To summarize the data collection step:
* We searched for all the currently available properties within a 5km radius around Geneva (this also includes properties in nearby towns).

* We extracted a number of key features that we considered to be useful for our later on study: 
    * 'cityName': city were the property is located. 
    * 'extraPrice': price of some utilities, e.g. heating (in CHF).
    * 'price': the total price of the rent, including 'extraPrice' (in CHF).
    * 'latitude': the latitued of the property.
    * 'longitude': the longitude of the property.
    * 'numberOfRooms': the number of rooms in the property.
    * 'street': the address of the property. 
    * 'surfaceLiving': the area of the property (in sqm).
    * 'zip': the postal code associated with the property.
  
* We checked the integrity of the data and found out that certain key features, e.g. price, or surface area, were missing for some items. We then decided to drop all entries with missing key features: numberOfRooms, surfaceLiving, and price. 

* We converted the type of each column to a more suitable type (e.g. 'zip' should be an integer), as by default, each feature was set to be an 'object' type.
* We added two additional columns containing useful feature for later analysis:
    * 'pricePerRoom': price per room, i.e 'price'/'numberOfRooms' (in CHF).
    * 'pricePerSqm': price per sqm, i.e 'price'/'surfaceLiving' (in CHF/sqm).


For indexing, our database uses the unique identyfier code assigned by immoscout24.ch, and looks like this:
![](Images_from_ScrapingImmoscout24ch_FA.ipynb/db_geneva_app_FA_df.png)





### Scraping foursquare: Geneva services distribution.
We leveraged the foursquare database to get insights on the organization of the city of Geneva, by identified how services that could influence the value of a property are distributed around the city are distributed around the city, such as the public transportation, hospitals and schools. This will help us answer questions such as: are certain areas more accessible than others? The code used in this analysis can be found in the Geneva_POI_FA.ipynb notebook.
In short, we queried the foursquare database looking for venues belonging to certain categories, and we created a data frame for each category. One example is the head of the dataframe where we stored the location of high schools, as whown below:

![](Images_from_Geneva_POI_FA/highschools_df.png)




### Scraping foursquare: venues per property.
To exploit the foursquare database, we decided to compile a list with the top venues in 500m radius around each property (the code for this can be found in GenevaLocations_FA.ipynb). This will enable us to answer questions like: are properties closer to the hottest nightlife spots more or less expensive? Is better accessibility to public transportation relevant in increasing housing prices?
We created a dictionary where the keys are the unique properties' identifiers as found in mmoscout24.ch. The item corresponding to each key is a dataframe storing informations on each venue.

Quering foursquare gives us access to the categry ID of a venue. The category ID generally corresponds to a very specific subcategory. For example, the category ID “Padangnese Restaurant', a regional cuisine of Indonesia, falls under the category of 'Indonesian Restaurant', which is also a subcategory of 'Asian Restaurant', and the most inclusive category 'Food. Because we are primarily interested in the more inclusive category of each venue, we needed to add this information to our dataframes. The auxiliary code for this can be found in the 'Scraping_foursquare_cat_FA.ipynb' notebook.

Finally, the dataframe containing a list of the nearby venues for a certain property will look like this:

![](Images_from_GenevaLocations_FA/Venues_per_property_df.png)





## 3. Methodology. <a name="methodology"></a>


### Price driving key features.
The starting point of our analysis is identifying the key features that influence the most the price of the properties in our database, and describing the relationship quantitatively. We do this by generating plots of the collected properties prices vs. feature. We will focus on: i) 'zip': to look for correlations between prices and neighborhoods; ii) 'numberOfRooms' and 'surfaceLiving', as we expect the size of a property to be key feature for determining its price.

##### Price vs zip
A boxplot representation of 'price' vs 'zip' (top-left panel, figure below) reveals a fairly large variation in the prices of properties sharing the same postal codes, compared to the distribution of prices among different postal coded regions. The 'pricePerSqm'(top-right panel, figure below) also appears to be homogeneous, implying that overall, no neighborhood is evidently more expensive then the other. An exception could be the 1216 postal code, which appears to have
a significantly larger 'pricePerSqm'. However, here the explanation lies in the fact that the 'pricePerSqm' is significantly higher for smaller properties (bottom-left panel, figure below) and most of the properties associated with this postal code have indeed low 'surfaceLiving' values (bottom-right panel, figure below), which explains the apparently larger 'pricePerSqm'. With the present sample size, it is not possible to establish whether the observed abundance of smaller properties in this neighborhood is an intrinsic feature or a momentary occurrence. Therefore, it is safer to assume an homogeneity in the 'pricePerSqm' distribution across all neighborhoods.

This means that a straightfoward recommendation system on a price-neighborhood base is not possible.



<table><tr>
<td> <img src="figures_ai/Price_vs_Zip.png" alt="Drawing" style="width: 350px;"/></td>
<td> <img src="figures_ai/Price_per_sqm_vs_Zip.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>


<table><tr>
<td> <img src="figures_ai/Price_per_Sqm_vs_number_of_rooms.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/Surface_Living_vs_Zip.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>


##### Price vs size
As we can observe here, the price of a property is strongly correlated with its size. Both 'numberOfRooms' and 'surfaceLiving' are good candidates for predicting the expected price of a property. In principle either factor could be used for drawing guidelines for our customers.  Our choice will be 'surfaceLiving' as the seemingly linear relationship with the price makes it the more straightforward predictor.
<img src="figures_ai/Draft_figures_Capstone-01.png" alt="Drawing" style="width: 500px;"/>


To achieve quantitative results, we performed a linear regression, after splitting the sample into train (left panel, figure below) and test sets (right panel, figure below).
The results show that linear relation between price and surface living is a satisfactory model, resulting in a R2-score of 0.77 as calculated on the test set. The starting price for a property (intercept) is 650CHF and each extra sqm costs about 27CHF.

<table><tr>
<td> <img src="figures_ai/train_linear_fit.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/test_linear_fit.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

We further exploited this analysis for screening out outliers. We defined the normalized deviation from expected price for a property as:

'price_dev' = ('expected_price'-'price')/'price',

and add this feature to our dataset. Then, we plot this quantity vs a feature of our choice, e.g. 'surfaceLiving'.
It is evident in the plot below (left panel) that some properties show absurd values. We can filter out these entries to obtain a more meaningful dataset (right panel).

<table><tr>
<td> <img src="figures_ai/price_dev.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/price_dev_outliers.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

### Properties and services distribution around Geneva.
According to a study looking at properties' prices in Shenzhen, China, published in the journal <a href="https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0164553">PLoS One</a> (2016; 11(10): e0164553), the proximity of certain services to a property can influence its price. Although the study was conducted outside of Europe, we can assume that its findings can be related to housing in Geneva since schools, hospitals, transports, parks, and markets all satisfy basic human needs across cultures. We will therefore include similar factors into our project.

The next step of our analysis is to get a visual representation of the distribution of the properties around Geneva and compare it to the distribution of said services. The results are shown in the figure below.

![](Images_from_ScrapingImmoscout24ch_FA.ipynb/properties2.png)| ![alt](Images_from_Geneva_POI_FA/transports2.png) |![alt](Images_from_Geneva_POI_FA/parks2.png)
- | - | -
![alt](Images_from_Geneva_POI_FA/hospitals2.png) | ![alt](Images_from_Geneva_POI_FA/markets2.png)|![alt](Images_from_Geneva_POI_FA/schools2.png) 



* Properties: As shown in the top-left panel, the properties gather around the city center, and we can identify few strategical spots: i) the Cornavin railway station (the main station of the city, red marker); ii) the main neighborhoods around the city  center (Plainpalais, Champel, and les Eaux-Vives marked by the yellow, brown , and purple markers, respectively); iii) the nearby town of Carouge (green marker), which borders Geneva and is well connected to the city center. For this reason, it is typically considered another large neighborhood of Geneva. Finally, a few properties are scattered in the outskirts of the main neighborhoods. 

* Transports: public transportation lines in Geneva (top-center panel) are more densely packed in the city center, with the highest density around the train station of Cornavin, where many bus and tram lines meet. From there, these lines branch out into different directions and finally reaches the outskirts of the city.

* Parks: Geneva is rich in green areas, with most of them inside the neighborhoods of Cornavin train station, Plainpalais and les Eaux-Vives.

* Hospitals: medical services can be found mainly between the Plainpalais and Champel neighborhoods, and the town of Carouge.

* Markets: markets are typically found all across the city, with a slightly higher density in the northern part.

* Schools: schools can also be found across the city, especially in the neighborhood of les Eaux-Vives.

This analysis helps us rationalize our previous observations about the homogeneous prices distribution across different areas. Geneva is a relatively small city with a high density of key services, that are well distributed across the the various neighborhoods. The ramified network of public transportation makes access to services quite easy, leveling the differences that one could expect between more central properties and marginal ones. 

### Clustering method

The last step of our analysis consisted of grouping properties according to the distributions of nearby venues. 

So far, we have shown that the prices of properties of similar sizes is homogeneous around the city and cannot be the determining factor for deciding where to live. Instead, the decision should be based on the desired accessible services nearby, or on the culture and atmosphere of the neighborhood, e.g. active nightlife spot, or a more quite place.

We therefore extended our database to include the number of venues that fall under certain macro-categories (e.g. food, nightlife hot spots, park, etc.) for each entry in our database and used these features for grouping into clusters.


<img src="figures_ai/Venues_info_per_property.png" alt="Drawing" style="width: 900px;"/>



In the course of this study, several attempts with different clustering methods were made. In particular, we tried to use density-based clustering methods extensively, as no initial guess on the number of group is needed here. However, the results were far from satisfying. Remarkably, no combination of input parameters was able to generate clusters significantly different to provide us a solid base for a recommendation system.

Finally, the simplest solution proved to be the most effective. We implemented a k-means clustering with kclusters = 3. The result is shown in the map below, in which the available properties in different colors is shown to correspond to different groups.

<img src="figures_ai/Clustered_properties_geneva.png" alt="Drawing" style="width: 800px;"/>


Clustering with the chosen features clusters where properties belonging to the same group form spatially localized bunches. This is not surprising, as close by properties will share similar distributions of close by venues.
We observe that cluster 0 (blue) comprises  properties i the outskirt;  cluster 1 (orange) is mainly localized in Plainpalais and around the train station, while cluster 2 (green) includes properties in Carouge, Les Eaux-Vives and around the station respectively.

By plotting the 'price_dev' per group, we see that different groups are not significantly different in this respect. Again, this is consistent with the lack of correlation between a property position in the city and its deviation from its expected average price. 



<img src="figures_ai/price_dev_clusters.png" alt="Drawing" style="width: 400px;"/>


## 4. Results. <a name="results"></a>


After performing the clustering of our dataset and classifying in 3 distinct group, we can finally get more insight on which features set aside each group. As we can see in the boxplots below, cluster 1 has the most venues in all 'Food', 'Nightlife Spot', and 'Outdoors & recreation' categories compared to cluster 2 and cluster 0. Cluster 2 ranks after cluster 1 in all three categories and finally cluster 0 scores the lowest in all categories. 

<table><tr>
<td> <img src="figures_ai/Clustered_food.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/Clustered_nightlifespots.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/Clustered_outdoors.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>




The same pattern, with slightly lower significance, can be observed for the remainign categories.

<table><tr>
<td> <img src="figures_ai/Clustered_travel.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/Clustered_arts_and_entertainment.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="figures_ai/Clustered_shop_and_services.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

## 5. Discussion. <a name="discussion"></a>

We can finally summarize  our results and deliver our recommendations to our customers.

The primary key message is that price across the city do not vary significantly for properties of similar dimensions. Furthermore, properties of any size appears to be evenly distributed around the city. Therefore, we can reassure our customer that when choosing a neighborhood, he or she would only need to consider the size of the property as well as the atmosphere of neighborhood.

Then, depending on the customer’s personal tastes, we would recommend properties belonging to different groups. Those who prefer an active neighborhood full of life and activities would be recommended group 1, i.e. the train station area and Plainpalais, since most of the city’s services and night life can be found here. Group 1 is also a good choice for those looking for a calmer area, but still close to the city center, since group 1 which is mainly located in the center of the city has areas where life is more quiet. 

Finally, for people who prefer an even more quite place, group 0 is the best choice. Living here will mean to be farther away from the city center. Luckily, the city’s excellent public transportation system connects residents with quick and efficient access to the city center. For those who prefer to travel by car, living in group 0 could save troubles related to traffic and parking since spaces are often limited in the center, amounting to long waiting times during rush hours. 


## 6. Conclusions. <a name="conclusions"></a>


In this project we developed a strategy for supporting newcomers arriving to Geneva in finding a suitable accommodation. This strategy acts as a guide, which enables us to help customers find a  place that fits with their taste and lifestyle, as well as help them to better invest their resources. For this, we scraped one of the most up to date rental listing website in Switzerland (immoscout24.ch) and we leveraged the foursquare database to deliver insights and targeted recommendations to our customers. 

We identified the 'livingSurface', i.e. the surface area, of a property as the best single indicator in predicting its expected price. This gives us the proper tool to help our customer evaluate whether certain location comes at a favorable price.

For available properties currently on the market, we ascertained that prices are mostly homogeneous across the city and that the choice of a location should be mainly based on the accessible services available in the area. 

Further development of this project could involve storing historical informations about the available properties, via regular scraping of the immoscout24.ch website. This would offer a more accurate description of the distributions of the properties in the city, possibly revealing insights that the present study did not detect due to the limited sample size. 

